In [1]:
from magenta.music.performance_lib import PerformanceEvent
import magenta.music as mm
import re

In [8]:
interactive_config = {}
interactive_config['steps_per_second'] = 100

# Interactive

This notebook holds interactive controls to explore inference models qualitatively, i.e. synthesize MIDI and look at pianoroll viz.

### Creating a bundle file
The [bundle format](https://github.com/tensorflow/magenta/blob/master/magenta/protobuf/generator.proto) is a convenient way of combining the model checkpoint, metagraph, and some metadata about the model into a single file. It is intended to make sharing pre-trained models easier. To generate a bundle, use the [create_bundle_file](https://github.com/tensorflow/magenta/blob/master/magenta/music/sequence_generator.py) method within SequenceGenerator.

# Environment Setup

Includes package installation for sequence synthesis. May take a few minutes.

This Jupyter is modified from [Magenta Performance RNN Colaboratory](https://colab.research.google.com/notebooks/magenta/performance_rnn/performance_rnn.ipynb?authuser=0#scrollTo=nzGyqJja7I0O).

**FILE DEPENDENCIES**

* Salamander piano SoundFont. Samples by Alexander Holm [[Link](https://archive.org/details/SalamanderGrandPianoV3)]. Converted to sf2 by John Nebauer [[Link](https://sites.google.com/site/soundfonts4u)]. You can fetch those from here (591.9 MiB):
    * `gsutil -m cp gs://download.magenta.tensorflow.org/soundfonts/Yamaha-C5-Salamander-JNv5.1.sf2 /tmp/`

In [9]:
# Hack to allow python to pick up the newly-installed fluidsynth lib.
import ctypes.util
def proxy_find_library(lib):
    if lib == 'fluidsynth':
        return 'libfluidsynth.so.1'
    else:
        return ctypes.util.find_library(lib)

ctypes.util.find_library = proxy_find_library

In [10]:
# mm.plot_sequence(sequence)

In [11]:
class InteractiveSess():
    def __init__(self):
        pass
    
    def _parse_seq(self, seq):
        """
        Args:
            A text-like string of events (i.e. "ON50 ON71 SHIFT50 OFF71")
        Returns:
            The text converted into a NoteSequence class type.
        """    
        events = seq.split()
        
        performance = mm.Performance(
          steps_per_second=interactive_config['steps_per_second'])
        
        for event in events:
            if re.match(r'^ON[0-9]+$', event):
                event_type = 1         
            elif re.match(r'^OFF[0-9]+$', event):
                event_type = 2
            elif re.match(r'^SHIFT[0-9]+$', event):
                event_type = 3
            else:
                raise ValueError('Unknown event type: %s' % event)
                
            event_value = int(re.search(r'[0-9]+', event).group(0))
            
            event = PerformanceEvent(event_type, event_value)
            performance.append(event)

        note_seq = performance.to_sequence()
        return note_seq
        
    
    def load(self, path):
        with open(path, 'r') as f:
            text_seqs = f.readlines()
            text_seqs = [l.strip() for l in text_seqs]
            text_seq = text_seqs[0]
            
            print('INFO: Found {} sequences. Using the first one. The rest are ignored.'.format(len(text_seqs)))
            
            note_seq = self._parse_seq(text_seq)
        return note_seq

In [12]:
path_to_sequence = './data/performance_seq_text/eval_arrangement_inputs.txt'

interactive = InteractiveSess()
sequence = interactive.load(path_to_sequence)

INFO: Found 11 sequences. Using the first one. The rest are ignored.


In [13]:
mm.plot_sequence(sequence)

# Try line below if above doesn't work
# mm.notebook_utils.plot_sequence(sequence)

Loading BokehJS ...

In [18]:
# mm.play_sequence(sequence, mm.midi_synth.fluidsynth,
#                  sf2_path='./tmp/Yamaha-C5-Salamander-JNv5.1.sf2')


# Synthesizes audio from a music_pb2.NoteSequence using a waveform.
# mm.midi_synth(sequence)

# Creates an interactive player for a synthesized note sequence.
# Lower sound quality than `fluidsynth` with a good SoundFont.
mm.notebook_utils.play_sequence(sequence)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [17]:
import fluidsynth
fs = fluidsynth.Synth()